In [ ]:
#
# Attempts to implement the following:
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from shapely import Polygon
from   udist_scatterplots_via_sectors import UDistScatterPlotsViaSectors, xyUniformSampleDistributionSectorTransformDEBUG, xyUniformSampleDistributionSectorTransform
import time
import rtsvg
rt = rtsvg.RACETrack()
import random
num_of_pts   = [10, 20, 10]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

#
# Simple Renderer
#
def renderSVG(xs, ys, colors, r=0.2):
    x0, y0, x1, y1 = min(xs), min(ys), max(xs), max(ys)
    xperc, yperc   = (x1-x0)*0.01, (y1-y0)*0.01
    x0, y0, x1, y1 = x0-xperc, y0-yperc, x1+xperc, y1+yperc
    svg = []
    svg.append(f'<svg x="0" y="0" width="256" height="256" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">')
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" x="0" y="0" fill="#ffffff" />')
    for i in range(len(xs)): svg.append(f'<circle cx="{xs[i]}" cy="{ys[i]}" r="{r}" fill="{colors[i]}" />')
    svg.append('</svg>')
    return ''.join(svg)
    
#x_new, y_new = xyJustVecs(_xvals_, _yvals_, _weights_, _colors_, iterations=80, vector_scalar=0.1)
#rt.tile([renderSVG(_xvals_, _yvals_, _colors_), renderSVG(x_new, y_new, _colors_)], spacer=10)

x_new, y_new, svgs, svgs_for_sectors, svg_animation, df_fine = xyUniformSampleDistributionSectorTransformDEBUG(rt, _xvals_, _yvals_, _weights_, _colors_, iterations=32)
udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=32)
#rt.table(svgs, per_row=8, spacer=10)
rt.tile([svg_animation, udspvs.animateIterations(animation_dur="4s")], spacer=10)

In [ ]:
def comparePointIteration(point_i=0, iteration=0):
    _df0_ = df_fine.filter((pl.col('iteration') == iteration) & (pl.col('point_i') == point_i) & (pl.col('s_wgt') > 0.0)).sort('s_wgt')
    _df1_ = udspvs.df_fully_filled[iteration].filter(pl.col('__index__') == point_i).sort('_w_sum_')
    if set(_df0_['s_wgt']) != set(_df1_['_w_sum_']): 
        raise Exception(f'Weight sums do not match {point_i=} {iteration=}')
    if set(_df0_['s_u']) != set(_df1_['anchor_u']): 
        l0 = sorted(list(_df0_['s_u']))
        l1 = sorted(list(_df1_['anchor_u']))
        for i in range(len(l0)):
            if l0[i] != l1[i]:
                raise Exception(f'Anchor U does not match {point_i=} {iteration=} {l0[i]=} {l1[i]=}')
        raise Exception(f'Anchor U does not match {point_i=} {iteration=}')
    if set(_df0_['s_v']) != set(_df1_['anchor_v']): 
        raise Exception(f'Anchor V does not match {point_i=} {iteration=}')

#for iteration in range(33):
#    print('.',end='')
#    for point_i in range(41):
#        comparePointIteration(point_i, iteration)

In [ ]:
iteration, point_i = 0, 0
df_fine.filter((pl.col('iteration') == iteration) & (pl.col('point_i') == point_i) & (pl.col('s_wgt') > 0.0)).sort('s_wgt')

In [ ]:
udspvs.df_fully_filled[iteration].filter(pl.col('__index__') == point_i).sort('_w_sum_')

In [ ]:
udspvs.df_uv[iteration].filter(pl.col('__index__') == point_i)

In [ ]:
df_fine.filter((pl.col('iteration') == iteration) & (pl.col('point_i') == point_i)).sort('s', descending=True)

In [ ]:
df_fine.filter((pl.col('iteration') == iteration) & (pl.col('point_i') == point_i)).sum()

In [ ]:
df = pl.DataFrame({'x':[1,2,3,4,1,2,3,4,1,4],
                   'y':[9,9,9,9,8,8,8,8,7,7],
                   'z':[2,3,1,3,5,2,3,5,2,3]})
df

In [ ]:
# Get all unique y values
unique_y = df['y'].unique().sort(descending=True)

# Create complete combinations of x (1,2,3,4) for each y
complete_combinations = []
for y_val in unique_y:
    for x_val in [1, 2, 3]:
        complete_combinations.append({'x': x_val, 'y': y_val})

complete_df = pl.DataFrame(complete_combinations)

# Left join to preserve existing z values and add missing combinations
result_df = (complete_df
             .join(df, on=['x', 'y'], how='left')
             .with_columns(pl.col('z').fill_null(0))
             .sort(['y', 'x'], descending=[True, False]))

print("\nDataframe with missing combinations filled:")
print(result_df)

In [ ]:
df = pl.DataFrame({'x':[1,5,10,11]})
y_values = [1,2]
df.join(pl.DataFrame({'y':y_values}), how='cross')

In [ ]:
df_all_sectors = udspvs.df_at_iteration_start[iteration].join(pl.DataFrame({'sector': [i for i in range(16)]}), how='cross').drop(['c','w'])
_df_           = udspvs.df_fully_filled[iteration]
_df_           = _df_.join(df_all_sectors, on=['__index__', 'x', 'y'], how='left') \
                     .with_columns(pl.col('_w_ratio_').fill_null(0.0))

_df_.filter(pl.col('__index__') == point_i).sort('sector')